# Como Utilizar a Função DAX `SUMMARIZECOLUMNS` no Power BI para Criar KPIs Poderosos

## Introdução

O Power BI é uma ferramenta poderosa para análise e visualização de dados, amplamente utilizada por profissionais de diversas áreas para criar relatórios dinâmicos e interativos. Uma das linguagens mais importantes no Power BI é o DAX (Data Analysis Expressions), que permite a criação de fórmulas complexas para manipulação de dados. Neste artigo, focaremos na função DAX `SUMMARIZECOLUMNS`, explorando suas funcionalidades e como utilizá-la para criar KPIs valiosos. Utilizaremos o dataset `financial-sample` disponível no [data.world](https://data.world/jcnok/financial-sample) e integraremos as visualizações diretamente a partir do Jupyter Notebook usando a biblioteca `powerbiclient`.

## Fórmula DAX Escolhida: `SUMMARIZECOLUMNS`

### Principais Funcionalidades

A função `SUMMARIZECOLUMNS` é utilizada para criar tabelas resumidas, agrupando dados por colunas especificadas e realizando agregações. Ela é particularmente eficiente e otimizada para consultas DAX no Power BI.

### Sintaxe

```dax
SUMMARIZECOLUMNS (
    <GroupBy_ColumnName>,
    [<FilterTable>],
    <Name> [, <Expression>] [, <Name>, <Expression>]...
)
```

### KPIs que Podem Ser Geradas

- Vendas Brutas por Segmento
- Descontos por Segmento
- Lucro Médio por Segmento
- Preço de Venda Médio
- Descontos Médios

### Quando e Por Que Usar

Use `SUMMARIZECOLUMNS` quando precisar criar resumos de dados agrupados por uma ou mais colunas, especialmente quando o desempenho é uma preocupação. É ideal para gerar KPIs que resumem dados de maneira eficiente.

## Antes vale a pena passar um overview das principais ferramentas que serão utilizadas:

### Plataforma data.world

A [data.world](https://data.world/) é uma plataforma colaborativa para dados e análises, que permite armazenar, compartilhar e explorar dados. Facilita a colaboração entre equipes e a integração de dados de várias fontes, promovendo insights e decisões baseadas em dados.

### Usabilidade da API data.world

A API [datadotworld](https://pypi.org/project/datadotworld/) permite acessar e manipular datasets diretamente em Python. Ela facilita a integração com ferramentas analíticas, possibilitando consultas, uploads e compartilhamento de dados. Ideal para projetos que exigem automação e análise de dados eficiente.

### Biblioteca powerbiclient

A biblioteca [powerbiclient](https://pypi.org/project/powerbiclient/) facilita a integração do Power BI com Jupyter Notebooks. Permite a inserção e interação com relatórios do Power BI diretamente em notebooks, promovendo uma análise de dados interativa e visual. Ideal para analistas de dados que desejam combinar o poder de visualização do Power BI com a flexibilidade dos notebooks Jupyter.

### API Power BI

A API Power BI permite automação e integração avançada com a plataforma Power BI. Ela possibilita criar, gerenciar e compartilhar relatórios e dashboards, além de executar consultas DAX e obter insights. Ideal para desenvolvedores que buscam automatizar processos e integrar visualizações de dados em suas aplicações, promovendo análises e decisões baseadas em dados.

## Passo a Passo: 5 Exemplos de `SUMMARIZECOLUMNS`

Vamos criar e visualizar cinco KPIs usando a função `SUMMARIZECOLUMNS` no Jupyter Notebook.

### Configurar o Ambiente

Certifique-se de ter as bibliotecas necessárias instaladas:

```
$ pip install pandas requests powerbiclient python-dotenv datadotworld
```

Instalar a Biblioteca python-dotenv
Primeiro, instale a biblioteca python-dotenv para carregar as variáveis de ambiente de um arquivo .env.

Crie um arquivo chamado .env no diretório raiz do seu projeto. Adicione suas chaves de acesso e tokens nesse arquivo. Por exemplo:

DW_API_TOKEN=seu_token_da_api_aqui

### Importar as Bibliotecas

In [22]:
import requests
import pandas as pd
from powerbiclient import Report, models, QuickVisualize, get_dataset_config
from powerbiclient.authentication import DeviceCodeLoginAuthentication
from dotenv import load_dotenv
import os
import datadotworld as dw

### Carregar o Arquivo .env no Seu Código Python
No início do seu script Python, use a biblioteca dotenv para carregar as variáveis de ambiente do arquivo .env.

In [2]:
# Carregar as variáveis do arquivo .env
load_dotenv()

# Acessar as variáveis de ambiente
dw_api_token = os.getenv('DW_API_TOKEN')

### Esta biblioteca requer um token de autenticação da API data.world para funcionar.
Seu token de autenticação pode ser obtido no data.world depois que você habilitar o [Python em Integrações]('https://data.world/integrations/python')

* **Para configurar a biblioteca, execute o seguinte comando:**

$ dw configure

* Alternativamente, os tokens podem ser fornecidos por meio da variável de ambiente DW_AUTH_TOKEN . Em máquinas MacOS ou Unix, execute (substituindo <YOUR_TOKEN>> abaixo pelo token obtido anteriormente):
`export DW_AUTH_TOKEN=<YOUR_TOKEN>`

`$ export DW_AUTH_TOKEN=dw_api_token`

In [3]:
!export DW_AUTH_TOKEN=dw_api_token

### Carregando o dataset via API:

* A função query() permite que conjuntos de dados sejam consultados ao vivo usando linguagens de consulta SQL ou SPARQL .

* Para consultar um conjunto de dados, invoque a função query() . Por exemplo:

In [8]:
results = dw.query('jcnok/financial-sample', 'SELECT * FROM financial_sample')

In [23]:
df = results.dataframe
df.head()

,segment,country,product,discount_band,units_sold,manufacturing_price,sale_price,gross_sales,discounts,sales,cogs,profit,date,month_number,month_name,year
0,Government,Canada,Carretera,None,1618.5,3,20,32370.0,0.0,32370.0,16185.0,16185.0,2014-01-01,1,January,2014
1,Government,Germany,Carretera,None,1321.0,3,20,26420.0,0.0,26420.0,13210.0,13210.0,2014-01-01,1,January,2014
2,Midmarket,France,Carretera,None,2178.0,3,15,32670.0,0.0,32670.0,21780.0,10890.0,2014-06-01,6,June,2014
3,Midmarket,Germany,Carretera,None,888.0,3,15,13320.0,0.0,13320.0,8880.0,4440.0,2014-06-01,6,June,2014
4,Midmarket,Mexico,Carretera,None,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2014-06-01,6,June,2014


### Autenticação no Power BI

In [24]:
device_auth = DeviceCodeLoginAuthentication()
token = device_auth.get_access_token()
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
}

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FBV27LTWA to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


### Visualizando seus Dados no Power BI

Agora que seus dados estão prontos e você está autenticado, é hora de criar visualizações no Power BI. O código a seguir faz exatamente isso:

In [25]:
PBI_visualize = QuickVisualize(get_dataset_config(df), auth=device_auth)

* **Renderizando o Relatório**

Agora já é possível renderizar o relatório diretamente no Jupyter Notebook:

In [26]:
PBI_visualize

QuickVisualize()

### Definir e Executar Consultas DAX

Defina as consultas DAX que serão utilizadas para extrair os KPIs.

In [12]:
dax_queries = [
    {"query": "EVALUATE SUMMARIZECOLUMNS('Table'[Segment], 'Table'[Gross Sales])"},
    {"query": "EVALUATE SUMMARIZECOLUMNS('Table'[Segment], 'Table'[Discounts])"},
    {"query": "EVALUATE SUMMARIZECOLUMNS('Table'[Segment], 'Table'[Profit])"},
    {"query": "EVALUATE ROW('Average Sale Price', CALCULATE(AVERAGE('Table'[Sale Price])))"},
    {"query": "EVALUATE ROW('Average Discounts', CALCULATE(AVERAGE('Table'[Discounts])))"}
]

dataset_id = 'edf35402-a994-4111-b4a3-65b644fb25de'
url = f'https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}/executeQueries'

def execute_dax_query(dax_query, headers, url):
    response = requests.post(url, headers=headers, json={"queries": [dax_query], "serializerSettings": {"includeNulls": True}})
    response_data = response.json()
    results = response_data['results'][0]['tables'][0]['rows']
    df = pd.DataFrame(results)
    return df

dfs = []
for dax_query in dax_queries:
    df = execute_dax_query(dax_query, headers, url)
    dfs.append(df)
```

### Passo 5: Visualizar os Resultados

Vamos utilizar `matplotlib` para criar visualizações e a biblioteca `QuickVisualize` para integrá-las ao Power BI.

```python
# Visualização de Vendas Brutas por Segmento
df_sales = dfs[0]
df_sales.columns = ['Segment', 'Gross Sales']

plt.figure(figsize=(10, 6))
plt.bar(df_sales['Segment'], df_sales['Gross Sales'])
plt.title('Vendas Brutas por Segmento')
plt.xlabel('Segmento')
plt.ylabel('Vendas Brutas')
plt.show()
```

```python
# Visualização de Descontos por Segmento
df_discounts = dfs[1]
df_discounts.columns = ['Segment', 'Discounts']

plt.figure(figsize=(10, 6))
plt.bar(df_discounts['Segment'], df_discounts['Discounts'], color='orange')
plt.title('Descontos por Segmento')
plt.xlabel('Segmento')
plt.ylabel('Descontos')
plt.show()
```

```python
# Visualização de Lucro Médio por Segmento
df_profit = dfs[2]
df_profit.columns = ['Segment', 'Profit']

plt.figure(figsize=(10, 6))
plt.bar(df_profit['Segment'], df_profit['Profit'], color='green')
plt.title('Lucro Médio por Segmento')
plt.xlabel('Segmento')
plt.ylabel('Lucro Médio')
plt.show()
```

```python
# Visualização de Preço de Venda Médio
df_avg_sale_price = dfs[3]
df_avg_sale_price.columns = ['Metric', 'Average Sale Price']

plt.figure(figsize=(10, 6))
plt.bar(df_avg_sale_price['Metric'], df_avg_sale_price['Average Sale Price'], color='purple')
plt.title('Preço de Venda Médio')
plt.xlabel('Métrica')
plt.ylabel('Preço de Venda Médio')
plt.show()
```

```python
# Visualização de Descontos Médios
df_avg_discounts = dfs[4]
df_avg_discounts.columns = ['Metric', 'Average Discounts']

plt.figure(figsize=(10, 6))
plt.bar(df_avg_discounts['Metric'], df_avg_discounts['Average Discounts'], color='red')
plt.title('Descontos Médios')
plt.xlabel('Métrica')
plt.ylabel('Descontos Médios')
plt.show()
```

### Integrando Visualizações no Power BI

Use a biblioteca `QuickVisualize` para integrar as visualizações ao Power BI.

```python
group_id = 'seu-group-id'  # Substitua pelo ID do seu workspace
report_id = 'seu-report-id'  # Substitua pelo ID do seu relatório

report = Report(group_id=group_id, report_id=report_id, auth=device_auth)
quick_viz = QuickVisualize(df_sales)
quick_viz.show()
```

### Explicação de Cada KPI

- **Vendas Brutas por Segmento**: Indica a receita bruta gerada por cada segmento. Importante para identificar quais segmentos são mais lucrativos.
- **Descontos por Segmento**: Mostra o total de descontos aplicados por segmento. Útil para avaliar políticas de desconto.
- **Lucro Médio por Segmento**: Mede o lucro médio obtido por segmento, ajudando a entender a rentabilidade de cada segmento.
- **Preço de Venda Médio**: Calcula o preço médio de venda dos produtos. Essencial para análises de precificação.
- **Descontos Médios**: Indica o desconto médio aplicado, importante para avaliar a eficácia das estratégias de desconto.

### Quando e Por Que Usar a Integração do Power BI ao Jupyter Notebook

A integração do Power BI ao Jupyter Notebook é útil para:

- **Análise Avançada**: Combinar as capacidades de análise do Python com as ferramentas de visualização do Power BI.
- **Automação de Relatórios**: Automatizar a geração de relatórios e dashboards.
- **Colaboração**: Facilitar a colaboração entre cientistas de dados e analistas de negócios.

### Conclusão

Neste artigo, exploramos a função DAX `SUMMARIZECOLUMNS` e como utilizá-la para criar KPIs poderosos no Power BI. Demonstramos como executar consultas DAX diretamente no Jupyter Notebook e integrar visualizações ao Power BI, proporcionando uma abordagem eficiente para análises avançadas e visualização de dados. Aproveite essas técnicas para melhorar suas habilidades em Power BI e criar dashboards cada vez mais sofisticados e informativos.

### Recursos Adicionais

- [Documentação Oficial do Power BI](https://docs.microsoft.com/power-bi/)
- [Tutoriais de DAX](https://docs.microsoft.com/dax/)
- [Biblioteca powerbiclient](https://pypi.org/project/powerbiclient/)

---

Espero que este artigo tenha sido útil e informativo. Boa sorte na sua jornada com Power BI e DAX!